In [88]:
import pandas as pd

df=pd.read_csv('invoicedetail.csv' , encoding='latin-1')

df=df[df.dateofcreation>='2020-06-10 12:55:59']

df=df.drop(['pid','brand','invoice_id.1','partner_id','customer_id','dateofcreation'],axis=1)

In [89]:
df

,name,invoice_id,quantity,discount,totalprice,grandtotal
20,Akurit 4 Tablet,3291,60,10.80,161.00,161.0
66,Crocin 1000mg Tablet,1734,3,10.00,12.29,670.0
67,Crocin 1000mg Tablet,1741,2,10.00,8.19,8.0
68,Crocin 1000mg Tablet,1742,1,10.00,4.10,4.0
69,Crocin 1000mg Tablet,1743,4,10.00,16.38,16.0
...,...,...,...,...,...,...
12540,Pure hands hand Sanitizer,6661,1,3.61,240.00,660.0
12541,Nivea Shower Gel fresh pure,6683,1,4.00,120.00,120.0
12542,Everyuth Naturals Exfoliating Walnut Scrub,6659,1,0.00,75.00,75.0
12543,chana dal loose,6662,500,0.00,35.00,35.0


In [90]:
# Removed Biased fields 
df=df[~df['name'].str.contains("Extras|Amul Gold Full Cream Milk|Amul Taaza Toned Milk|Amul Masti Dahi|Amul Chach Buttermilk|Parle G biscuit 5 Rs|Parle 20 20 Cashew Cookie|Loose Sugar",na=False)]

In [91]:
# Invoice
df['invoice_id'].nunique()

4027

In [92]:
# Brands
df['name'].nunique()

842

In [93]:
df_purchase=df.groupby('invoice_id').name.apply(list).reset_index()
df_purchase

,invoice_id,name
0,1237,[Viman Jeera]
1,1241,"[Haldiram Navrattan Namkeen, Haldiram Aloo Bhu..."
2,1242,"[Catch club soda with spring water, Uncle Chip..."
3,1243,[Harvest Gold Fruit Bread]
4,1244,"[Amul Kool Kesar Flavour, Kurkure Puffcorn Yum..."
...,...,...
4022,6684,[Harvest Gold white Bread Sabki Bread Sabki Pa...
4023,6685,"[Mix Dal, Haldiram Punjabi Tadka Namkeen, Hald..."
4024,6686,[White Eggs]
4025,6687,[Eagle Premium haldi Powder]


In [94]:
# Finding the most frequent occurrences of pairs in a list of lists

sell_products=[]
for sell in df_purchase['name']:
    sell_products.append(sell)

sell_products

[['Viman Jeera'],
 ['Haldiram Navrattan Namkeen',
  'Haldiram Aloo Bhujia',
  'Cadbury Dairy Milk Chocolate'],
 ['Catch club soda with spring water', 'Uncle Chips Spicy Treat'],
 ['Harvest Gold Fruit Bread'],
 ['Amul Kool Kesar Flavour', 'Kurkure Puffcorn Yummy Cheese  Namkeen'],
 ['Soft Drink', 'Lays Classic Salted Chips'],
 ['Maaza Refresh mango Drink'],
 ['Britannia Cake Gobbles Choco Chill', 'Amul Pure Ghee', 'Cadbury Gems'],
 ['Catch club soda with spring water',
  'Panasonic Jumbo cell- AA',
  'White Eggs'],
 ['Harvest Gold Brown Bread Hearty Brown'],
 ['Frooti Mango drink'],
 ['Britannia Cake Gobbles Choco Chill',
  'Britannia Cake Gobbles Fruity fun',
  'Parle G Gold Biscuits',
  'Dettol original liquid Hand Wash '],
 ['Harvest Gold white Bread Sabki Bread'],
 ['MDH Haldi Powder'],
 ['Harvest Gold white Bread Sabki Bread'],
 ['Mother Dairy Classic Dahi Cup'],
 ['Britannia Cake Gobbles Fruity fun',
  'Britannia Cake Gobbles Choco Chill',
  'Roll yo choco Swiss roll',
  'Ajanta B

In [95]:
# Finding the most frequent occurrences of pairs in a list of lists
from collections import Counter
from itertools import combinations

d  = Counter()
for sub in sell_products:
    if len(sell_products) < 2:
        continue
    for comb in combinations(sub,2):
        d[comb] += 1

# print(d.most_common())

In [96]:
for elem in d.most_common():
    print(elem) 

(('Frooti Mango drink', 'Bisleri with added minerals drinking water'), 19)
(('Mountain Dew Soft Drink', 'Bisleri with added minerals drinking water'), 14)
(('Fortune Premium Kachi Ghani', 'Fortune Refined  Soya Bean Oil'), 13)
(('Kurkure Masala Munch', 'Lays American Style Cream & Onion Chips'), 12)
(('Loose Masoor Dal', 'Loose Moong Dhuli Dal'), 12)
(('Fortune Premium Kachi Ghani', 'Aashirvaad Shudh Chakki Atta'), 12)
(('Fortune Premium Kachi Ghani', 'TATA Salt'), 12)
(('Loose Arhar/Toor Dal', 'Loose Masoor Dal'), 11)
(('Mountain Dew Soft Drink', 'Frooti Mango drink'), 11)
(('TATA Salt', 'Fena Superwash detergent powder Rose & Chandan '), 10)
(('Fena Superwash detergent powder Rose & Chandan ', 'Fortune Refined  Soya Bean Oil'), 10)
(('Soft Drink', 'Lays Classic Salted Chips'), 9)
(('Harvest Gold white Bread Sabki Bread', 'Amul Pasteurised Butter'), 9)
(('Lays American Style Cream & Onion chips', 'Lays India Magic Masala Chips'), 9)
(('Soft Drink', 'Frooti Mango drink'), 9)
(('Fortune

(('Scotch Brite Scrub Pad S-shape ', 'Santoor Gentle Hand Wash Refill (sweet lime peel & Tulsi)'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Atta Biscuits munshi Ram'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Maggi Noodles 4 Pack'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Ramdev Strong Hing'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Rin Big Bar (250 GM Each)'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Ananda Desi Ghee'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Fortune Premium Kachi Ghani'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'kala Chana'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'kabuli chana chole'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Moong Dhuli dal'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Dhania Sabut loose'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Dove Cream beauty bathing bar'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Vim dishwash liquid with power of lemons'), 1)
(('Scotch Brite Scrub Pad S-shape ', 'Rajma White/Chitra'), 1)
(('Scotch Brite Scrub Pad S-shape '

(('Good Day Butter Cookies', 'Dettol original liquid Hand Wash '), 1)
(('Good Day Butter Cookies', 'Rajdhani Poha'), 1)
(('Good Day Butter Cookies', 'Taj Mahal Tea'), 1)
(('Good Day Butter Cookies', 'MDH Amchur Powder'), 1)
(('Good Day Butter Cookies', 'MDH Garam Masala'), 1)
(('Good Day Butter Cookies', 'MDH Haldi Powder'), 1)
(('Good Day Butter Cookies', 'MDH Dhania Powder '), 1)
(('Good Day Butter Cookies', 'MDH Meat Masala'), 1)
(('Good Day Butter Cookies', 'MDH Chicken Masala'), 1)
(('Good Day Butter Cookies', 'MDH Rajmah Masala'), 1)
(('Good Day Butter Cookies', 'Rajdhani Sooji Coarse granules'), 1)
(('Good Day Butter Cookies', 'Rajdhani Dalia'), 1)
(('Good Day Butter Cookies', 'Alpenliebe Pop- Cream stawberry flavour'), 1)
(('Good Day Butter Cookies', 'Sulphur free Sugar'), 1)
(('Good Day Butter Cookies', 'MDH Sambhar Masala'), 1)
(('Good Day Butter Cookies', 'Maggi Rich tomato ketchup free worth rs 60  ketchup'), 1)
(('Good Day Chocochip Cookies', 'Dettol original liquid Hand W

(('Tiger Krunch chocochips biscuit', 'Aashirvaad Shudh Chakki Atta'), 1)
(('Mountain Dew Soft Drink', 'Mountain Dew Soft Drink'), 1)
(('Catch club soda with spring water', 'cadbury dairy milk thank you choclate Rs40'), 1)
(('I Shakti Iodised Salt', 'OM Suji Toast Crispy Rusk'), 1)
(('I Shakti Iodised Salt', 'Haldiram Bhujia'), 1)
(('OM Suji Toast Crispy Rusk', 'Haldiram Bhujia'), 1)
(('Fair & lovely advancd multi vitamin face cream', 'Mangaldeep Puja Agarbattis Mogra'), 1)
(('Frooti Mango drink', 'haldiram bhujia'), 1)
(('Doritos SWEET CHILLI FLAVOUR', 'Doritos Sweet Chilli Flavour'), 1)
(('Mangaldeep Puja Agarbattis Mogra', 'ACT II Butter Flavour Crispy Crunch Popcorn'), 1)
(('Mangaldeep Puja Agarbattis Mogra', 'Haldiram Boondi'), 1)
(('ACT II Butter Flavour Crispy Crunch Popcorn', 'Haldiram Boondi'), 1)
(('Harvest Gold white Bread Sabki Bread', 'Doritos Nacho Cheese Flavour'), 1)
(('haldiram Classic nut cracker', 'Mother Dairy Classic Dahi Cup'), 1)
(('haldiram Classic nut cracker', 

In [97]:
# for a in combinations(['a','b','c'],2):
#     print(a)

In [11]:
# D=Counter()



In [12]:
# D['Aakash']=5

In [98]:
# D

In [14]:
# D['Aakash']=D['Aakash']+1

In [99]:
# D

In [100]:
df_number_occurence=pd.DataFrame([d])

In [102]:
# df_number_occurence

In [104]:
df_number_occurence=df_number_occurence.T

In [105]:
df_number_occurence[df_number_occurence[0]>10].to_csv('two_item_combo.csv')


In [106]:
# df_number_occurence[df_number_occurence[0]>5]

In [107]:
# d

In [108]:
# d3 = Counter()
# for sub in sell_products:
#     if len(sell_products) < 3:
#         continue
#     for comb in combinations(sub,3):
#         d3[comb] += 1

In [109]:
# d3

In [24]:
# df_number_occurence3=pd.DataFrame([d3])

In [110]:
df_number_occurence=pd.DataFrame([d])

In [111]:
# df_number_occurence3
# df_number_occurence3=df_number_occurence3.T

In [27]:
# df_number_occurence3[df_number_occurence3[0]>4].to_csv('three_item_combo.csv')

In [112]:
# df_number_occurence3[df_number_occurence3[0]>4]

In [113]:
# d4 = Counter()
# for sub in sell_products:
#     if len(sell_products) < 4:
#         continue
#     for comb in combinations(sub,4):
#         d4[comb] += 1

In [114]:
# df_number_occurence4=pd.DataFrame([d4])

In [115]:
# df_number_occurence4=df_number_occurence4.T
# df_number_occurence4


In [116]:
# df_number_occurence4[df_number_occurence4[0]>3].to_csv('four_item_combo.csv')

In [117]:
# df_number_occurence4[df_number_occurence4[0]>3]

In [118]:
# d

In [35]:


# interest_pairs= [a for a in d.most_common() if a[1]>min_freq_threshold]

# interest_pairs
    

In [119]:
import numpy as np

n= df['name'].nunique()
produts= list(df['name'].unique())

data_structure = np.zeros([n, n], dtype = int)

data_structure

for pair in d.most_common():
    i=produts.index(pair[0][0])
    j=produts.index(pair[0][1])
    data_structure[i,j]=pair[1]
    data_structure[j,i]=pair[1]

data_structure

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [120]:
min_freq_threshold = 3

query='Frooti Mango drink'
ind=produts.index(query)
for i in np.where(data_structure[ind,:] >= min_freq_threshold)[0]:
    print(produts[i],data_structure[ind,i])

Fortune Refined  Soya Bean Oil 6
Rajdhani Besan Gram flour Grade-1 3
Maggi 2 Minutes noodles 5
Soft Drink 9
Coca Cola softdrink 3
Coca Cola- softdrink 3
Fortune Premium Kachi Ghani 8
Good Day Butter Cookies 6
Thums up Soft Drink 3
Mountain Dew Soft Drink 11
Harvest Gold white Bread Sabki Bread 5
Lays American Style Cream & Onion chips 3
Kurkure Puffcorn yummy cheese 3
Lays American Style Cream & Onion Chips 8
Loose Basmati rice Tukda 5
TATA Salt 7
ACT II Butter Flavour Crispy Crunch Popcorn 4
Fena Superwash detergent powder Rose & Chandan  4
Haldiram takatak chatpata masala namkeen 3
Shyam G Pasta crunchy & crispy 3
Maaza original mango Drink 8
Britannia Vitarich sandwich Premium Ehite Bread 3
Britannia Vitarich sandwich Premium White Bread 3
Haldiram Bhujia 4
Loose Arhar/Toor Dal 3
Bisleri with added minerals drinking water 19
Mountain Dew Soft Drink New 4
Priyagold Butter delite Namkeen butter biscuits  4


In [122]:
# for col in
import csv
with open('innovators.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for col in produts:
        ind=produts.index(col)
        interestIndex=np.where(data_structure[ind,:] >= min_freq_threshold)[0]
        interestList=[col]
        for i in np.where(data_structure[ind,:] >= min_freq_threshold)[0]:
            interestList.append(produts[i])
        print(interestList)
        writer.writerow(interestList)

['Akurit 4 Tablet']
['Crocin 1000mg Tablet']
['Crocin 120 Suspension Strawberry']
['Volini 100mg Tablet SR']
['Little Hearts Biscuits']
['Powder - Haldi']
['Bambino Vermicelli Roasted']
['Fortune Refined  Soya Bean Oil', 'Rajdhani Sooji Coarse granules', 'MDH Garam Masala', 'Britannia Marie Gold Biscuits', 'Maggi 2 Minutes noodles', 'Soft Drink', 'Lays India Magic Masala chips', 'Coca Cola softdrink', 'Mountain Dew Soft Drink', 'Milkfood Rich desi Ghee', 'MDH Meat Masala', 'Kurkure Masala Munch', 'Frooti Mango drink', 'TATA Salt', 'Everest Sabji Masala', 'shiv pujari jeera', 'Eagle Premium haldi Powder', 'Fena Superwash detergent powder Rose & Chandan ', 'Ghadi detergent Cake', 'Vim dishwash Bar with power of lemons Pack of 4', 'Britannia Vitarich sandwich Premium Ehite Bread', 'Loose Arhar/Toor Dal', 'Loose Masoor Dal', 'Loose Rajma', 'Bisleri with added minerals drinking water']
['Tata PremiumTea desh ki chai']
['Dettol Original Soap (75gm Each)', 'Aashirvaad Shudh Chakki Atta', 'Mou

IndexError: index 842 is out of bounds for axis 0 with size 842